In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [74]:
import pandas as pd
import numpy as np

In [75]:
# bring in data
#path = '/Users/reneehall/Documents/Learning/lighthouse_labs/coursework-lighthouse-labs/Mid-term project/colab_data/'
flights = pd.read_csv('mini_sample.csv')


In [76]:
flights.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled', 'diverted',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'flights',
       'distance', 'origin_fl_density', 'dest_fl_density', 'day', 'month',
       'year', 'weekday', 'day_of_year', 'crs_arr_hour', 'crs_dep_hour',
       'timestamp', 'crs_arr_daytime', 'crs_dep_daytime',
       'origin_airport_size', 'dest_airport_size', 'season', 'dist_group',
       'dep_early_morning', 'mean_op_carrier_delay',
       'mean_delay_origin_airport', 'mean_delay_dest_airport',
       'mean_dep_delay_origin', 'mean_dep_delay_dest', 'predicted_speed',
       'percent

In [77]:
# assign X and y
cols=['day', 'month', 'weekday', 'op_unique_carrier', 
      'origin', 'dest',
           'crs_arr_daytime', 'crs_dep_daytime', 
           'origin_airport_size', 'dest_airport_size', 
      'dist_group', 'season', 'dep_early_morning',
      'percent_delay_dep', 'dist_group_delay_ratio']
# 'mkt_carrier_fl_num', 'op_carrier_fl_num', 'mkt_unique_carrier',

flights['arr_delay_flag'] = flights['arr_delay'].map(lambda x: 1 if x > 0 else 0)

X = flights[cols]
y = flights['arr_delay_flag']

In [78]:
X = pd.get_dummies(X, columns=['op_unique_carrier','origin','dest',
                               'crs_arr_daytime','crs_dep_daytime',
                               'month','weekday','day',
                               'dist_group','season','dep_early_morning',
                               'origin_airport_size','dest_airport_size'])

X.head()

,percent_delay_dep,dist_group_delay_ratio,op_unique_carrier_9E,op_unique_carrier_9K,op_unique_carrier_AA,op_unique_carrier_AS,op_unique_carrier_AX,op_unique_carrier_B6,op_unique_carrier_C5,op_unique_carrier_CP,op_unique_carrier_DL,op_unique_carrier_EM,op_unique_carrier_EV,op_unique_carrier_F9,op_unique_carrier_G4,op_unique_carrier_G7,op_unique_carrier_HA,op_unique_carrier_KS,op_unique_carrier_MQ,op_unique_carrier_NK,op_unique_carrier_OH,op_unique_carrier_OO,op_unique_carrier_PT,op_unique_carrier_QX,op_unique_carrier_UA,op_unique_carrier_VX,op_unique_carrier_WN,op_unique_carrier_YV,op_unique_carrier_YX,op_unique_carrier_ZW,origin_ABE,origin_ABI,origin_ABQ,origin_ABR,origin_ABY,origin_ACK,origin_ACT,origin_ACV,origin_ACY,origin_ADK,...,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,dist_group_1,dist_group_2,dist_group_3,dist_group_4,dist_group_5,dist_group_6,dist_group_7,dist_group_8,dist_group_9,dist_group_10,dist_group_11,season_1,season_2,season_3,season_4,dep_early_morning_0,dep_early_morning_1,origin_airport_size_1,origin_airport_size_2,origin_airport_size_3,origin_airport_size_4,dest_airport_size_1,dest_airport_size_2,dest_airport_size_3,dest_airport_size_4
0,-0.004614,0.123097,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1
1,2.000000,0.123097,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1
2,-2.000000,0.120592,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1
3,0.666667,0.123097,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1
4,0.333333,0.123097,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1


In [79]:
# make train-test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xsc = scaler.fit_transform(X)

In [80]:
#params for gridsearch
params = [
    {
        'n_estimators': [50, 100, 300, 500],
        'max_depth': [2,3,4,5, 6, 7]
    }
]

X_train, X_test, y_train, y_test = train_test_split(Xsc, y, train_size=0.7, test_size=0.3, random_state=101)

#Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# make the folds
k_folds = 5
# grid search model
grid = GridSearchCV(estimator=RandomForestClassifier(), param_grid=params, cv=k_folds, verbose=1, n_jobs=-1)
# fit model
grid_result = grid.fit(X_train, y_train)



Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 16.8min finished


In [81]:
best_accuracy = grid_result.best_score_
best_ntrees = grid_result.best_params_['n_estimators']
best_depth = grid_result.best_params_['max_depth']
print(f'The best model achieved an accuracy of: {best_accuracy}')
print(f'n_estimators: {best_ntrees}')
print(f'max_depth: {best_depth}')

best_model = grid_result.best_estimator_

The best model achieved an accuracy of: 0.6819142857142857
n_estimators: 100
max_depth: 7


In [82]:
y_pred=best_model.predict(X_test)

In [87]:
import pickle
#file_path = '/Users/reneehall/Documents/Learning/lighthouse_labs/coursework-lighthouse-labs/Mid-term project/lighthouse-mid-term-project/working-copy-files/renee-working-copies/pickles/'
filename = 'RF_CV1'
pickle.dump(best_model, open(filename, 'wb'))

In [83]:
#Evaluate
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, precision_recall_curve
from sklearn.metrics import average_precision_score,roc_auc_score, confusion_matrix, plot_confusion_matrix, precision_score
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
#print("F1 Score:", f1_score(y_test, y_pred))
#print("Recall:", recall_score(y_test, y_pred))
#print("Precision:", precision_score(y_test, y_pred))

from sklearn import metrics

cnf_matrix = metrics.classification_report(y_test,y_pred)
print(cnf_matrix)

              precision    recall  f1-score   support

           0       0.64      0.80      0.71      7429
           1       0.74      0.55      0.63      7571

    accuracy                           0.68     15000
   macro avg       0.69      0.68      0.67     15000
weighted avg       0.69      0.68      0.67     15000

Accuracy: 0.6768666666666666
              precision    recall  f1-score   support

           0       0.64      0.80      0.71      7429
           1       0.74      0.55      0.63      7571

    accuracy                           0.68     15000
   macro avg       0.69      0.68      0.67     15000
weighted avg       0.69      0.68      0.67     15000



In [46]:
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn import preprocessing

pca = PCA()
components = pca.fit_transform(Xsc)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
)




In [50]:
n_components = 2000

pca = PCA(n_components=n_components)
X_transformed = pca.fit_transform(Xsc)



